# Calculus & Gradients — FAANG-Level Lab

**Goal:** Implement and verify gradients like an ML engineer.

Key idea: *If you can’t gradient-check it, you don’t really trust it.*


In [1]:
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 1 — Finite Differences (Gradient Checking)

### Task 1.1: Implement numerical gradient for scalar f(w)

# HINT:
- Use central difference: (f(w+eps e_i) - f(w-eps e_i)) / (2 eps)

**Explain:** Why is central difference more accurate than forward difference?

In [ ]:
# Central difference is more accurate than forward difference because its error term is second-order 𝑂(ℎ^2), while forward difference has only first-order error 𝑂(ℎ).

In [3]:
def numerical_grad(f, w, eps=1e-5):
    # it takes input f, it takes 'w' (point at which i want Gradient)
    # numerical gradient of f at w
    w = np.asarray(w, dtype = float) # numpy array of floats
    # print('w float : ', w)
    g = np.zeros_like(w) # partial derivative of f -> same shape as w
    # print('g : ', g)

    for i in range(w.size):
        e = np.zeros_like(w)
        e[i] = 1.0 # for ith position, value is 0 -> basis vector
        # print('e : ', e)
        g[i] = (f(w + eps*e) - f(w - eps*e)) / (2 * eps) # Central difference: more effective than forward diff
        # print('g[i] : ', g[i])
    return g

# sanity test: f(w)=sum(w^2) => grad=2w
w = rng.standard_normal(5)
print('w : ', w)
f = lambda v: float(np.sum(v*v))
g_num = numerical_grad(f, w)
g_true = 2*w
check('grad_close', np.allclose(g_num, g_true, atol=1e-6))

w :  [ 0.36159505  1.30400005  0.94708096 -0.70373524 -1.26542147]
OK: grad_close


## Section 2 — Chain Rule in Code

### Task 2.1: Gradient of MSE for linear model

Model: y_hat = Xw
Loss: L(w) = (1/n) * sum_i (y_hat_i - y_i)^2

# HINT:
- Let r = Xw - y
- grad = (2/n) * X^T r

**FAANG gotcha:** shape mismatches; keep w as (d,) and X as (n,d).

In [5]:
def mse_loss_and_grad(X, y, w):
    r = X @ w - y                   # gives residual prediction error vector using linear model
    n = X.shape[0]                  # number of samples used for normalization
    loss = float(np.mean(r * r))    # Compute mean squared error loss
    grad = (2.0/n) * (X.T @ r)      # Apply grad chain rule
    return loss, grad


n, d = 20, 4
X = rng.standard_normal((n, d))     # Generate input feature matrix
w = rng.standard_normal(d)          # Initialize weight vector
y = rng.standard_normal(n)          # Generate target values
loss, grad = mse_loss_and_grad(X, y, w) # computing MSE loss and analytic gradient

# gradient check
f = lambda v: mse_loss_and_grad(X, y, v)[0]
g_num = numerical_grad(f, w)
check('mse_grad', np.allclose(grad, g_num, atol=1e-5))
print('loss', loss)

OK: mse_grad
loss 7.244391404204653


## Section 3 — Logistic Regression (Core Interview Gradient)

### Task 3.1: Binary cross-entropy gradient

Given labels y in {0,1}.
p = sigmoid(Xw)
Loss = -(1/n) * sum(y log p + (1-y) log(1-p))

# HINT:
- sigmoid(z)=1/(1+exp(-z))
- grad = (1/n) * X^T (p - y)
- add numerical stability for logs (clip p)


In [7]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))         # apply sigmoid to map real values to probabilities (0,1)

def logreg_loss_and_grad(X, y, w):
    z = X @ w       # linear part of our model
    p = sigmoid(z)  # help make a prediction probability
    eps = 1e-12     # binary probability
    p_clip = np.clip(p, eps, 1.0 - eps)         # clip: to avoid log(0) during loss computation
    loss = float(-np.mean(y * np.log(p_clip) + (1 - y) * np.log(1 - p_clip)))  # Compute binary cross-entropy (log loss)
    grad = (X.T @ (p - y)) / X.shape[0]         # sum contributions from all features (Gradient via chain rule)
    return loss, grad     


n, d = 50, 3
X = rng.standard_normal((n, d))
w = rng.standard_normal(d)
y = rng.integers(0, 2, size=n)
loss, grad = logreg_loss_and_grad(X, y, w)      # compute logistic loss and analytic gradient

f = lambda v: logreg_loss_and_grad(X, y, v)[0]  # define loss-only function for gradient checking
g_num = numerical_grad(f, w, eps=1e-5)          # compute numerical gradient using finite differences

check('logreg_grad', np.allclose(grad, g_num, atol=1e-5))
print('loss', loss)

OK: logreg_grad
loss 0.834822923472775


## Section 4 — Jacobian/Hessian Intuition

### Task 4.1: Compute Hessian of f(w)=sum(w^2) numerically

# HINT:
- Hessian of sum(w^2) is 2I
- Use numerical_grad on each component of grad

This is mainly about *shape thinking*: Hessian is (d,d).

In [8]:
def numerical_hessian(f, w, eps=1e-4):
    w = np.asarray(w, dtype=float)  # float numpy input array
    d = w.size                      # number of parameters (dimension of input)
    H = np.zeros((d, d))            # initializing empty Hessian matrix (d×d)

    for i in range(d):              # loop over each dimension to compute second derivatives
        def fi(v):                  # define function representing partial derivative w.r.t ith variable
            e = np.zeros(d); e[i] = 1.0     # create unit vector for i-th direction
            return (f(v + eps * e) - f(v - eps*e)) / (2 * eps)  # compute central difference approximation of first derivative
        
        H[i, :] = numerical_grad(fi, w, eps = eps)  # compute gradient to obtain i-th row of Hessian (second derivatives)
    return H                        # return full Hessian matrix

w = rng.standard_normal(4)          # generate random parameter vector
f = lambda v: float(np.sum(v*v))    # define quadratic scalar function
H = numerical_hessian(f, w)         # Hessian of f at w

check('H_shape', H.shape == (4,4))
check('H_close', np.allclose(H, 2*np.eye(4), atol=1e-3))
print(H)

OK: H_shape
OK: H_close
[[ 1.99999999e+00 -2.22044605e-08  2.22044605e-08  0.00000000e+00]
 [-2.22044605e-08  1.99999999e+00  0.00000000e+00  0.00000000e+00]
 [ 2.22044605e-08  0.00000000e+00  1.99999999e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.99999999e+00]]


---
## Submission Checklist
- All TODOs completed
- Gradient checks pass
- Explain prompts answered
